# **0. 참고자료**  
## **0-1. 도서**

## **0-2. 논문, 학술지**
## **0-3. 웹 사이트**
- 꿈 많은 사람의 이야기 : 파이썬과 함께 추천 시스템 이해하기 기본편 - content based filtering | [[블로그 링크]](https://lsjsj92.tistory.com/565?category=1066953) 
- Today I Learned. : 추천 시스템 - 컨텐츠 기반 필터링이란? (CBF) | [[블로그 링크]](https://western-sky.tistory.com/43)
- 비둘기 둥지 : [ML / 추천 시스템] 1. 추천 시스템에 대해 알아봅시다. | [[블로그 링크]](https://dove-nest.tistory.com/29)

## **0-4. 데이터셋 출처**
- Kaggle : Games of All Time from Metacritic | [[데이터 셋 링크]](https://www.kaggle.com/datasets/xcherry/games-of-all-time-from-metacritic)

## **0-5. 이미지 출처**

# **1. 컨텐츠 기반 필터링 (Contents Based Filtering)**
## **1-1. 컨텐츠 기반 필터링 이란**   
- 컨텐츠 기반 필터링(이하, CBF)는 사용자가 과거에 좋아했던 컨텐츠를 파악하고 비슷한 아이템을 추천해 주는 방식이다.  
    e.g.) 음식을 예로 들면 어떤 유저가 배달 어플을 통해 'KFC'와 '맥도날드'에 좋은 평점을 주었다면,  
          같은 햄버거 프랜차이즈인 '롯데리아'를 추천해 줄 수 있다.  
- 유저가 좋아한 아이템을 뽑아낸 목록을 Item Profile이라 하고,  
  Item Profile로부터 공통된 특징을 뽑아낸 결과를 User Profile이라고 한다.  
  e.g.) Item Profile | 'KFC', '맥도날드', '버거킹', ...  
          User Profile | '햄버거', '프랜차이즈', '패스트푸드', ...
          
## **1-2. CBF의 장점**

1. 다른 유저의 데이터를 필요로 하지 않는다.
2. 개인의 취향을 고려한 추천이 가능하다.
3. 새로운 아이템이나 대중적이지 않은 아이템도 추천할 수 있다.
4. 사용자에게 추천하는 이유에 대해 설명할 수 있다.
   → User Profile을 뽑아낼 때 구성한 Feature들로 설명가능하다.

⚠️ 1,2,3번으로 인해 얻을 수 있는 장점은 다음과 같다.  
  - 새로운 아이템이 추가되었을 때, 평가한 사함이 없어 추천이 어려운 Cold Start Problem에서 자유로워진다.  
  - 개인의 평가에 기반하기 때문에 모든 유저들이 모든 아이템에 대해 평가하지 않는 Sparsity Problem에서  
    자유로워 진다.
  - 아무도 평가하지 않은 새로운 아이템과 인기 없는 아이템도 해당 아이템들의 Feature들만 뽑아낼 수 있다면,  
    협업 필터링 구현시 발생하는 No first-rater Problem에서 자유로워진다.
    
## **1-3. CBF의 단점**

1. Feature를 뽑아내기 어려운 데이터들이 존재한다.  
   → 음악 같은 경우 장르가 너무 다양하기 때문에 공통된 Feature를 찾기 어렵다.
2. 처음 유입된 유저에 대해 추천할 수 없다.  
   → 주변 유저 평가는 영향을 받지 않지만, 아예 처음 유입된 유저는 그에 대한 데이터가 존재하지 않아 추천할 수 없다.
   
# **2. 실제 데이터 셋을 이용해 구현해보자**
- 평소에 게임을 좋아해서 캐글에 있는 Metacritic 리뷰 데이터 셋을 가져왔다.  
⚠️ Kaggle API를 이용해 데이터를 다운로드 받는 방법은 다음과 같다.
1. kaggle에서 API를 발급 받는다.
2. pip install kaggle을 통해 kaggle api 패키지를 설치한다.
3. 홈 디렉토리에 숨김폴더로 kaggle폴더를 만들고 kaggle API키를 복사한다.
4. kaggle API의 권한을 변경한다.

In [2]:
## kaggle API 사용을 위한 패키지 설치
!pip install kaggle

## json 파일 업로드 확인
!ls -1ha ~/project/TIL/api/kaggle.json

## 캐글 숨김 폴더 생성
!mkdir -p ~/.kaggle

## 토큰 파일 옮기고, 권한 부여
!cp  ~/project/TIL/api/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

/home/jovyan/project/TIL/api/kaggle.json


In [3]:
## 데이터 셋 다운로드 받기
!kaggle datasets download -d xcherry/games-of-all-time-from-metacritic

## 파일 옮기고 압축풀기
!mkdir -p ~/project/TIL/AI_study/dataset/metacritic
!mv games-of-all-time-from-metacritic.zip ~/project/TIL/AI_study/dataset/zips/games-of-all-time-from-metacritic.zip
!unzip ~/project/TIL/AI_study/dataset/zips/games-of-all-time-from-metacritic.zip -d ~/project/TIL/AI_study/dataset/metacritic

 83%|███████████████████████████████▍      | 2.00M/2.42M [00:00<00:00, 4.58MB/s]
100%|██████████████████████████████████████| 2.42M/2.42M [00:00<00:00, 4.60MB/s]
Archive:  /home/jovyan/project/TIL/AI_study/dataset/zips/games-of-all-time-from-metacritic.zip
replace /home/jovyan/project/TIL/AI_study/dataset/metacritic/games_of_all_time.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [1]:
import pandas as pd
import numpy as np
import swifter
import ast

## **2-1. 데이터를 확인해보고, EDA 비스무리 하게 해보자**
- csv 포맷으로 되어있는 데이터 셋을 불러온다.
- 데이터 셋의 칼럼은 총 9개로 구성되어 있었으며, 아래처럼 구성되어 있었다.
|column|설명|
|:--|:--|
|game_name|게임 이름|
|meta_score|메타 크리틱 점수|
|user_score|유저 점수|
|platform|게임 플랫폼|
|description|게임에 관한 내용|
|url|메타크리틱 페이지 내 게임 링크|
|developer|개발사|
|genre|게임 장르|
|type|싱글 게임인지 멀티 게임인지|
|rating|게임 등급|
- 이 중에서 게임 장르를 User profile로 사용하여 게임을 추천하도록 하였다.  
⚠️ 추천할 게임의 등급은 굳이 필요없을 것 같아 게임 추천 데이터에서 제거하였다.

In [2]:
DATASET_PATH = '~/project/TIL/AI_study/dataset/metacritic'

csv = pd.read_csv(f'{DATASET_PATH}/games_of_all_time.csv')
csv = csv.iloc[:, :-1]
csv.head(3)

,game_name,meta_score,user_score,platform,description,url,developer,genre,type
0,The Legend of Zelda: Ocarina of Time,99.0,91.0,['nintendo-64'],"As a young boy, Link is tricked by Ganondorf, ...",https://www.metacritic.com/game/nintendo-64/th...,Nintendo,"['Action Adventure', 'Fantasy']",singleplayer
1,Super Mario Galaxy,97.0,91.0,['wii'],[Metacritic's 2007 Wii Game of the Year] The u...,https://www.metacritic.com/game/wii/super-mari...,Nintendo,"['Action', 'Platformer', '3D']",singleplayer
2,Super Mario Galaxy 2,97.0,91.0,['wii'],"Super Mario Galaxy 2, the sequel to the galaxy...",https://www.metacritic.com/game/wii/super-mari...,Nintendo EAD Tokyo,"['Action', 'Platformer', '3D']",singleplayer


In [3]:
csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8831 entries, 0 to 8830
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   game_name    8831 non-null   object 
 1   meta_score   8831 non-null   float64
 2   user_score   8831 non-null   float64
 3   platform     8831 non-null   object 
 4   description  8831 non-null   object 
 5   url          8831 non-null   object 
 6   developer    8821 non-null   object 
 7   genre        8827 non-null   object 
 8   type         6727 non-null   object 
dtypes: float64(2), object(7)
memory usage: 621.1+ KB


- 총 8831개의 row에서 genre, developer, type column에는 더 적은 개수의 row가 있는것으로 보아 결측치를 처리해 주었다.
    - genre column을 이용해서 CBF를 구현해 볼 것이기 때문에, genre column에서 결측치는 제거해 주었다.
    - developer, type column은 개발사와 싱글, 멀티 플레이어인지 나타내는 데이터이기 때문에 그냥 결측치를 'unknown'으로 채워젔다.

In [4]:
csv = csv[csv['genre'].notna()]
csv['developer'] = csv['developer'].fillna('unknown')
csv['type']      = csv['type'].fillna('unknown')
csv.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8827 entries, 0 to 8830
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   game_name    8827 non-null   object 
 1   meta_score   8827 non-null   float64
 2   user_score   8827 non-null   float64
 3   platform     8827 non-null   object 
 4   description  8827 non-null   object 
 5   url          8827 non-null   object 
 6   developer    8827 non-null   object 
 7   genre        8827 non-null   object 
 8   type         8827 non-null   object 
dtypes: float64(2), object(7)
memory usage: 689.6+ KB


- 이번 주제를 공부하며 아주 신기한 패키지 두 가지를 알게 되었다.
    1. swifter          | pandas에서의 DataFrame 처리 속도 향상을 위한 병렬 처리 패키지
    2. ast.literal_eval | 엄밀히 말하면 패키지가 아니라 함수이지만, 문자열 안의 리스트, 딕셔너리를 진짜 리스트, 딕셔너리로 바꿔주는 함수

- swifter는 좀 더 자세히 실험할 내용과 설명이 필요하여 다음에 따로 적어두겠다.

In [32]:
## ast.literal_eval() 함수 예시
expr = '[1, 2, 3, 4, 5, 6, 7]'
print(f'before | {type(expr)}  ,  이거 리스트니? | {type(expr) == list}')

eval_list = ast.literal_eval(expr)
print(f'after  | {type(eval_list)} ,  이거 리스트니? | {type(eval_list) == list}\n')

## 정말 리스트로 변한 녀석이라 리스트에서 하는 모든 것들을 할 수 있다.
print(f'인덱싱을 해보자 | {eval_list[5]}, {eval_list[3]} \n슬라이싱도 가능 | {eval_list[2:5]}')

before | <class 'str'>  ,  이거 리스트니? | False
after  | <class 'list'> ,  이거 리스트니? | True

인덱싱을 해보자 | 6, 4 
슬라이싱도 가능 | [3, 4, 5]


In [41]:
## 리스트와 동일하게 문자열 안에 딕셔너리 형태로 있으면, 딕셔너리로 변한다.
expr = '{1 : "dove", 2 : "pigeon", 3 : "hippo", 4 : "turtle"}'
print(f'before | {type(expr)}  ,  이거 딕셔너리니? | {type(expr) == dict}')

eval_dict = ast.literal_eval(expr)
print(f'after  | {type(eval_dict)} ,  이거 딕셔너리니? | {type(eval_dict) == dict}\n')

## 마찬가지로 딕셔너리에서 하는 모든 것들을 할 수 있다.
print(f'key로 value를 가져와보자 | {eval_dict[3]}, {eval_dict[2]} \nitems()도 가능         | {eval_dict.items()}')

before | <class 'str'>  ,  이거 딕셔너리니? | False
after  | <class 'dict'> ,  이거 딕셔너리니? | True

key로 value를 가져와보자 | hippo, pigeon 
items()도 가능         | dict_items([(1, 'dove'), (2, 'pigeon'), (3, 'hippo'), (4, 'turtle')])


- swifter와 ast.literal_eval을 이용하여 genre와 platform에 있는 칼럼을 전처리 해주었다.

In [5]:
csv['genre'] = csv['genre'].swifter.apply(lambda x: ' '.join(ast.literal_eval(x)))
csv['platform'] = csv['platform'].swifter.apply(lambda x: set(ast.literal_eval(x)))

Pandas Apply:   0%|          | 0/8827 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/8827 [00:00<?, ?it/s]

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
vectorizer = CountVectorizer(ngram_range = (1, 3))
genre_vecs = vectorizer.fit_transform(csv['genre'])
genre_sims = cosine_similarity(genre_vecs, genre_vecs).argsort()[:, ::-1]
genre_vecs.shape, csv['genre'].shape

((8827, 3013), (8827,))

In [8]:
tf_idf     = TfidfVectorizer(stop_words = 'english')
tf_idf_mat = tf_idf.fit_transform(csv['description'])
des_sim = cosine_similarity(tf_idf_mat, tf_idf_mat).argsort()[:, ::-1]

tf_idf_mat.shape, csv['description'].shape

((8827, 32326), (8827,))

In [9]:
def filtering(name, sims, top = 15):
    
    game_idx = csv[csv['game_name'] == name].index.values
    sim_idx  = sims[game_idx, :top].reshape(-1)
    
    sim_idx  = sim_idx[sim_idx != game_idx]
    result   = csv.iloc[sim_idx].sort_values(['meta_score', 'user_score'], ascending = [False, False])
    
    return result

- 실제 서비스처럼 쿼리를 입력받아 동작할 수 있도록 함수를 만들어 보았다.

In [10]:
def advanced_filtering(sims, top=15, **query):
    
    ## 입력받은 게임이름과 유사도 행렬을 이용해 한 번 필터링한다.
    df = filtering(query['name'], sims)
    
    
    platform, p_include  = query['platform']
    developer, d_include = query['developer']
    
    ## 인자값으로 딕셔너리에 저장되어 있는 값들을 이용해 데이터를 필터링 해준다.
    df = df[df['type'] == query['type']]
    
    ## 좀 더 깔끔하게 짜고 싶다..
    if (p_include == True) and (d_include == True): 
        df = df[df['platform'].apply(lambda x: platform in x)]
        df = df[df['developer'].apply(lambda x: developer in x)] 
        
    elif (p_include == False) and (d_include == True): 
        df = df[df['platform'].apply(lambda x: platform not in x)]
        df = df[df['developer'].apply(lambda x: developer in x)] 
        
    elif (p_include == True) and (d_include == False):
        df = df[df['platform'].apply(lambda x: platform in x)]
        df = df[df['developer'].apply(lambda x: developer not in x)]
        
    else:
        df = df[df['platform'].apply(lambda x: platform not in x)]
        df = df[df['developer'].apply(lambda x: developer not in x)]
    
    return df
    

## **2-3. 비교**

In [11]:
csv[csv['game_name'] == 'Super Mario Odyssey']

,game_name,meta_score,user_score,platform,description,url,developer,genre,type
4,Super Mario Odyssey,97.0,89.0,{switch},New Evolution of Mario Sandbox-Style Gameplay....,https://www.metacritic.com/game/switch/super-m...,Nintendo,Action Platformer 3D,singleplayer


In [12]:
filtering('Super Mario Odyssey', genre_sims).head(3)

,game_name,meta_score,user_score,platform,description,url,developer,genre,type
7613,Skylanders Swap Force,83.5,58.0,"{wii-u, playstation-4, playstation-3, xbox-360}",Embark on an exciting new adventure with the S...,https://www.metacritic.com/game/wii-u/skylande...,Vicarious Visions,Action Platformer 3D,unknown
7618,Sly 3: Honor Among Thieves,83.0,89.0,{playstation-2},Sly 3: Honor Among Thieves entices a broad ran...,https://www.metacritic.com/game/playstation-2/...,Sucker Punch,Action Platformer 3D,multiplayer
7614,Skylanders: Spyro's Adventure,79.5,52.5,"{playstation-3, 3ds}","The small purple dragon returns, bringing with...",https://www.metacritic.com/game/3ds/skylanders...,Vicarious Visions,Action Platformer 3D,unknown


In [13]:
filtering('Super Mario Odyssey', des_sim).head(3)

,game_name,meta_score,user_score,platform,description,url,developer,genre,type
29,Super Mario Advance 4: Super Mario Bros. 3,94.0,89.0,{game-boy-advance},"A combination of exciting platforming action, ...",https://www.metacritic.com/game/game-boy-advan...,Nintendo,Action Platformer 2D,multiplayer
64,Paper Mario,93.0,90.0,{nintendo-64},Mario pals around in an all-new action adventu...,https://www.metacritic.com/game/nintendo-64/pa...,Intelligent Systems,Role-Playing Console-style RPG,unknown
47,Super Mario 3D World,93.0,89.0,{wii-u},Work together with your friends or compete fo...,https://www.metacritic.com/game/wii-u/super-ma...,Nintendo,Platformer 3D Action,singleplayer


In [14]:
query = {
            'name' : 'Super Mario Odyssey',
            'type' : 'singleplayer',
             'platform' : ['pc', False], 
            'developer' : ['Nintendo', True]
        }


advanced_filtering(sims = genre_sims, top = 15, **query)

,game_name,meta_score,user_score,platform,description,url,developer,genre,type


In [15]:
advanced_filtering(sims = genre_sims, top = 15, **query)

,game_name,meta_score,user_score,platform,description,url,developer,genre,type


In [16]:
advanced_filtering(sims = des_sim, top = 15, **query)

,game_name,meta_score,user_score,platform,description,url,developer,genre,type
47,Super Mario 3D World,93.0,89.0,{wii-u},Work together with your friends or compete fo...,https://www.metacritic.com/game/wii-u/super-ma...,Nintendo,Platformer 3D Action,singleplayer
85,Super Mario Sunshine,92.0,84.0,{gamecube},"When Mario, Peach, and an entourage of Peach's...",https://www.metacritic.com/game/gamecube/super...,Nintendo,Action Platformer 3D,singleplayer
162,Mario Kart DS,91.0,87.0,{ds},Race and battle with up to eight karts at once...,https://www.metacritic.com/game/ds/mario-kart-ds,Nintendo,Driving Racing Arcade Kart Automobile,singleplayer
210,Super Mario 3D Land,90.0,84.0,{3ds},Get Ready for a New Spin on a Familiar Tail. S...,https://www.metacritic.com/game/3ds/super-mari...,Nintendo,Action Platformer 3D,singleplayer
673,Super Mario 3D All-Stars,82.0,64.0,{switch},Play three of Mario’s greatest 3D platform adv...,https://www.metacritic.com/game/switch/super-m...,Nintendo,Miscellaneous Compilation,singleplayer
1624,Mario & Luigi: Paper Jam,76.0,74.0,{3ds},Paper Mario jumps out of a book and into the b...,https://www.metacritic.com/game/3ds/mario-luig...,Nintendo,Role-Playing Japanese-Style,singleplayer
